In [9]:
import ast
import os
import pickle as pkl

import numpy as np
import pandas as pd
import torch
from torchmetrics.functional.classification import (
    multilabel_average_precision,
)

**Load annotations and results**

In [10]:
# model name
model_name = "slow_r50-w-negatives"
folder_path = "../dataset/results/"
metadata_file = "../dataset/metadata/metadata.csv"
behavioural_labels_file = "../dataset/metadata/behaviours.txt"
segements_file = "../dataset/metadata/segements.txt"

# list all result files in the folder which end with .pkl and contain the model name
result_info = {}

for file in os.listdir(folder_path):
    if file.endswith(".pkl") and model_name in file:
        is_kinetics = False

        if "-kinetics" not in file:
            epoch = file.split("_")[-2].split("=")[1]
        else:
            is_kinetics = True

            epoch = file.split("_")[-2].split("=")[1].split("-")[0]

        # get the split from the file name
        split = file.split("=")[-1].split(".")[0]

        # add the file to the dictionary
        # add model to the dictionary
        if model_name not in result_info:
            result_info[model_name] = {}
        # add epoch to the dictionary
        if epoch not in result_info[model_name]:
            result_info[model_name][epoch] = {}
        if split not in result_info[model_name][epoch]:
            result_info[model_name][epoch][split] = {}
        result_info[model_name][epoch][split] = {
            "file_path": os.path.join(folder_path, file),
            "is_kinetics": is_kinetics,
        }

In [11]:
result_info

{'slow_r50-w-negatives': {'100': {'val': {'file_path': '../dataset/results/model=slow_r50-w-negatives_e=100_split=val.pkl',
    'is_kinetics': False},
   'train': {'file_path': '../dataset/results/model=slow_r50-w-negatives_e=100_split=train.pkl',
    'is_kinetics': False}},
  '200': {'train': {'file_path': '../dataset/results/model=slow_r50-w-negatives_e=200_split=train.pkl',
    'is_kinetics': False},
   'val': {'file_path': '../dataset/results/model=slow_r50-w-negatives_e=200_split=val.pkl',
    'is_kinetics': False}},
  '0': {'train': {'file_path': '../dataset/results/model=slow_r50-w-negatives_e=0-kinetics_split=train.pkl',
    'is_kinetics': True},
   'val': {'file_path': '../dataset/results/model=slow_r50-w-negatives_e=0-kinetics_split=val.pkl',
    'is_kinetics': True}}}}

In [12]:
metadata_df = pd.read_csv(metadata_file)

with open(behavioural_labels_file, "rb") as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

with open(segements_file, "rb") as f:
    segments = [seg.decode("utf-8").strip() for seg in f.readlines()]


def read_files(model_results, epoch):
    with open(model_results[epoch]["train"]["file_path"], "rb") as f:
        train_data = pkl.load(f)

    with open(model_results[epoch]["val"]["file_path"], "rb") as f:
        val_data = pkl.load(f)

    return train_data, val_data


def results2df(train_data, val_data, metadata_df):
    # Process subclips
    subclips = []
    for i, split in enumerate([train_data, val_data]):
        for name, pred, feat, label in zip(
            split["names"], split["preds"], split["feats"], split["labels"]
        ):
            subclips.append(
                {
                    "name": name,
                    "split": i,
                    "pred": pred,
                    "feat": feat,
                    "negative": True if sum(label) == 0 else False,
                }
            )

    df = pd.DataFrame(subclips, columns=["name", "split", "pred", "feat", "negative"])

    df["split"] = df.split.map({0: "train", 1: "val"})
    df = df.merge(metadata_df, how="left", left_on="name", right_on="subject_id")

    # Apply sigmoid to predictions
    df["pred"] = df.pred.apply(lambda x: torch.sigmoid(torch.tensor(x)))

    # Convert label from str to int
    df.label = df.label.apply(lambda x: np.array(ast.literal_eval(x)))

    # Add negative
    df["negative"] = df.label.apply(lambda x: sum(x) == 0)

    # Add global location count to dataframe
    df["location_count"] = df.utm.map(df.utm.value_counts())

    # Return train and val dataframes
    train_df = df[df.split == "train"]
    val_df = df[df.split == "val"]

    return train_df, val_df


def print_per_segement_performance(map, segment, show_per_class=False):
    res = []
    for i, (b, s) in enumerate(zip(map, segments)):
        if s == segment:
            res.append({behaviours[i]: b})
    agg_values = []
    for r in res:
        for _, value in r.items():
            agg_values.append(value)
    # if show_per_class:
    #    print(f"{segment}: {np.mean(agg_values):.2f} {res}")
    # else:
    #    print(f"{segment}: {np.mean(agg_values):.2f}")

    if show_per_class:
        return {
            segment: {
                "mean": np.round(np.mean(agg_values), 2),
                "values": res,
            }
        }
    else:
        return {
            segment: {
                "mean": np.round(np.mean(agg_values), 2),
            }
        }

In [13]:
def calculate_metrics(df, round_to=2):
    # Train performance
    map = multilabel_average_precision(
        torch.tensor(np.stack(df["pred"])),
        torch.tensor(np.stack(df["label"])),
        num_labels=14,
        average="none",
    )
    map_head = print_per_segement_performance(map, "head")
    map_tail = print_per_segement_performance(map, "tail")
    map_fs = print_per_segement_performance(map, "few_shot")

    map_head = round(float(map_head["head"]["mean"]), round_to)
    map_tail = round(float(map_tail["tail"]["mean"]), round_to)
    map_fs = round(float(map_fs["few_shot"]["mean"]), round_to)

    avg_map = round(map.mean().item(), round_to)

    return avg_map, map_head, map_tail, map_fs

In [14]:
result_file = "results.csv"

for m in result_info:
    for epoch in result_info[m]:
        # print(f"Loading results for model: {m}, epoch: {epoch}")
        train_data, val_data = read_files(result_info[model_name], epoch)
        train_df, val_df = results2df(train_data, val_data, metadata_df)

        train_map, train_map_head, map_tail, map_fs = calculate_metrics(train_df)
        val_map, val_map_head, val_map_tail, val_map_fs = calculate_metrics(val_df)

        # Write results to file # with columns: model, epoch, train_map, train_map_head, train_map_tail, train_map_fs, val_map, val_map_head, val_map_tail, val_map_fs
        with open(result_file, "a") as f:
            f.write(
                f"{m},{epoch},{train_map},{train_map_head},{map_tail},{map_fs},{val_map},{val_map_head},{val_map_tail},{val_map_fs}\n"
            )